In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

from sklearn.cluster import KMeans

import wrangle


In [2]:
# acquire data
zillow_data = wrangle.get_zillow17_data()

Let me get that for you...


In [3]:
""
zillow_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    77381 non-null  int64  
 1   id                            77381 non-null  int64  
 2   parcelid                      77381 non-null  int64  
 3   airconditioningtypeid         24953 non-null  float64
 4   architecturalstyletypeid      206 non-null    float64
 5   basementsqft                  50 non-null     float64
 6   bathroomcnt                   77381 non-null  float64
 7   bedroomcnt                    77381 non-null  float64
 8   buildingclasstypeid           15 non-null     float64
 9   buildingqualitytypeid         49672 non-null  float64
 10  calculatedbathnbr             76772 non-null  float64
 11  decktypeid                    614 non-null    float64
 12  finishedfloor1squarefeet      6023 non-null   float64
 13  c

Take aways:
- pulled all tables from the database
- aproximatly 77,381 rows of data
- no duplicates based on parcelid and trasaction date


In [4]:
# prep data
zillow = wrangle.prepare_zillow(zillow_data)

In [5]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67678 entries, 0 to 77380
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            67678 non-null  int64  
 1   parcelid                      67678 non-null  int64  
 2   bathroomcnt                   67678 non-null  float64
 3   bedroomcnt                    67678 non-null  float64
 4   calculatedfinishedsquarefeet  67678 non-null  float64
 5   fips                          67678 non-null  float64
 6   fullbathcnt                   67678 non-null  float64
 7   latitude                      67678 non-null  float64
 8   longitude                     67678 non-null  float64
 9   lotsizesquarefeet             67678 non-null  float64
 10  propertycountylandusecode     67678 non-null  object 
 11  propertylandusetypeid         67678 non-null  float64
 12  rawcensustractandblock        67678 non-null  float64
 13  r

In [6]:
zillow

,id,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,fullbathcnt,latitude,longitude,lotsizesquarefeet,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,propertylandusedesc,county
0,1727539,14297519,3.5,4.0,3100.0,6059.0,3.0,33634931.0,-117869207.0,4506.0,...,485713.0,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,0.025595,2017-01-01,Single Family Residential,Orange
1,1387261,17052889,1.0,2.0,1465.0,6111.0,1.0,34449266.0,-119281531.0,12647.0,...,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13,0.055619,2017-01-01,Single Family Residential,Ventura
2,11677,14186244,2.0,3.0,1243.0,6059.0,2.0,33886168.0,-117823170.0,8432.0,...,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13,0.005383,2017-01-01,Single Family Residential,Orange
3,2288172,12177905,3.0,4.0,2376.0,6037.0,3.0,34245180.0,-118240722.0,13038.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,Single Family Residential,Los Angeles
4,1970746,10887214,3.0,3.0,1312.0,6037.0,3.0,34185120.0,-118414640.0,278581.0,...,73681.0,119407.0,2016.0,45726.0,1533.89,6.037124e+13,0.006940,2017-01-01,Condominium,Los Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77376,673515,11000655,2.0,2.0,1286.0,6037.0,2.0,34245368.0,-118282383.0,47405.0,...,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,0.020615,2017-09-20,Single Family Residential,Los Angeles
77377,2968375,17239384,2.0,4.0,1612.0,6111.0,2.0,34300140.0,-118706327.0,12105.0,...,50683.0,67205.0,2016.0,16522.0,1107.48,6.111008e+13,0.013209,2017-09-21,Single Family Residential,Ventura
77378,1843709,12773139,1.0,3.0,1032.0,6037.0,1.0,34040895.0,-118038169.0,5074.0,...,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,0.037129,2017-09-21,Single Family Residential,Los Angeles
77379,1187175,12826780,2.0,3.0,1762.0,6037.0,2.0,33937685.0,-117996709.0,6347.0,...,140000.0,522000.0,2016.0,382000.0,6317.15,6.037503e+13,0.007204,2017-09-25,Single Family Residential,Los Angeles


Take aways:
- Home types include 'Single Family', 'Mobile', 'Manufactured', 'Townhouse', 'Cluster', and 'Condominium' 
- outliers were removed from 'calculatedfinishedsquarefeet', 'bedroomcnt', and 'bathroomcnt'
- missing values were handled by removing all columns and rows that had more than 50% missing values.
- Columns dropped were 'heatingorsystemtypeid', 'buildingqualitytypeid', 'propertyzoningdesc', 'unitcnt', 'heatingorsystemdesc', 'Unnamed: 0'
- impute missing values useing the most frequent value in 'calculatedbathnbr', 'fullbathcnt', 'regionidcity', 'regionidzip', 'yearbuilt', and 'censustractandblock'.
- impute missing values using median in 'finishedsquarefeet12', 'lotsizesquarefeet', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'landtaxvaluedollarcnt', and 'taxamount'.
- created county column using fips
- only select properties with atleast 1 bedroom and 500 sqft
- 67678 rows of data
- 12.5% data loss...acceptable

In [7]:
# feature engineering
zillow = wrangle.feature_engineering(zillow)

In [8]:
zillow.head()

,id,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,fullbathcnt,latitude,longitude,lotsizesquarefeet,...,transactiondate,propertylandusedesc,county,sqft_per_bed,sqft_per_bath,total_rooms,bed_bath_rooms_per_sqft,age_in_years,taxrate,dollars_per_sqft
0,1727539,14297519,3.5,4.0,3100.0,6059.0,3.0,33634931.0,-117869207.0,4506.0,...,2017-01-01,Single Family Residential,Orange,775.000000,885.714286,7.5,0.002419,19.0,0.010763,330.090968
1,1387261,17052889,1.0,2.0,1465.0,6111.0,1.0,34449266.0,-119281531.0,12647.0,...,2017-01-01,Single Family Residential,Ventura,732.500000,1465.000000,3.0,0.002048,50.0,0.012225,316.723549
2,11677,14186244,2.0,3.0,1243.0,6059.0,2.0,33886168.0,-117823170.0,8432.0,...,2017-01-01,Single Family Residential,Orange,414.333333,621.500000,5.0,0.004023,55.0,0.011488,454.366854
3,2288172,12177905,3.0,4.0,2376.0,6037.0,3.0,34245180.0,-118240722.0,13038.0,...,2017-01-01,Single Family Residential,Los Angeles,594.000000,792.000000,7.0,0.002946,47.0,0.012247,61.087121
4,1970746,10887214,3.0,3.0,1312.0,6037.0,3.0,34185120.0,-118414640.0,278581.0,...,2017-01-01,Condominium,Los Angeles,437.333333,437.333333,6.0,0.004573,53.0,0.012846,91.011433


Take aways:
- created new columns for sqft per bed and bath
- new column beds + baths = total rooms
- created new column for sqft per room
- created new column for age of property
- created new column for tax rate
- created new column for tax amount per sqft


In [9]:
# split data
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle.split_zillow(zillow, 'logerror')

In [10]:
# check splits
print(train.shape)
print(validate.shape)
print(test.shape)

(37899, 35)
(16243, 35)
(13536, 35)


In [11]:
print(X_train.shape)
print(y_train.shape)
print(X_validate.shape)
print(y_validate.shape)
print(X_test.shape)
print(y_test.shape)

(37899, 34)
(37899,)
(16243, 34)
(16243,)
(13536, 34)
(13536,)


Take aways:
- Data was split into train(75%), validate(15%), and test(10%)
- Target(y) varible is 'logerror'
- Features(X) are all except 'logerror'
- All shapes are valid